In [1]:
#importing the needed libraries
from sklearn.datasets import load_iris
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV, cross_validate, KFold
import numpy as np
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.model_selection import KFold
from sklearn.metrics import roc_auc_score
import numpy as np
from sklearn.model_selection import LeaveOneGroupOut
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_validate
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import f1_score
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import roc_auc_score, make_scorer, recall_score, f1_score, confusion_matrix, accuracy_score

In [2]:
#reading in the data we will use in the machine learning pipeline, with the top ten features of the method
df = pd.read_excel('5fs_impedance_training_set_10_final.xlsx')
df

,ImP_exp_diff_mean,ImP_exp_dur_sd,ImP_RRV_RMSSD,ImP_insp_flow_sd,ImP_ie_ratio_sd,ImP_duty_cycle_mean,ImP_insp_amp_sd,ImP_exp_amp_sd,ImP_RSP_Symmetry_PeakTrough,ImP_ie_ratio_mean,Task_Label,Participant,Classification
0,0.118857,0.171499,382.434308,15663.415593,0.048638,0.489158,50985.092943,46774.304708,0.517348,0.958792,12.0a,10785,0
1,0.225143,0.173505,390.361005,20811.126431,0.069124,0.485452,30734.204708,31615.925634,0.527694,0.945842,12.0b,10785,0
2,0.236000,0.222699,434.669530,16277.918645,0.050025,0.476758,31615.118946,26435.238020,0.520548,0.912526,12.0c,10785,0
3,0.250667,0.190379,403.425334,15714.039237,0.071244,0.483507,29114.145601,25312.370962,0.530733,0.938632,12.0d,10785,0
4,0.140800,0.174310,483.103302,6162.134521,0.124387,0.481444,43005.468164,28574.347285,0.527149,0.936589,12.0e,10785,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2569,0.240000,0.274684,318.223192,3850.325373,0.265283,0.489451,8019.865362,14865.886766,0.523436,0.988002,85,98586,0
2570,0.708000,0.764264,1236.868088,72599.234334,0.375168,0.427189,121971.615000,102499.398673,0.511816,0.809380,87,98586,1
2571,0.119556,0.122849,159.285907,4549.907724,0.082769,0.499069,6323.609559,12526.615103,0.543171,0.999821,89,98586,0
2572,1.567500,0.980966,2899.130116,21555.712739,0.546540,0.482030,47869.716992,51080.627080,0.472444,1.070194,91,98586,1


In [3]:
# extracting the features from the DataFrame
features = df.drop(columns=['Classification', 'Participant', 'Task_Label'])

# extracting the Classification (0/1) and participants from the dataframe
target = df['Classification'].values
participants = df['Participant'].values
task = df['Task_Label'].values

# extracting the feature names
feature_names = features.columns.tolist()

# converting the features, target, participants to numpy arrays
features_array = features.values
target_array = target

# creating the dictionary with the needed format to enter into the nested cross-validation
data_dict = {
    'data': features_array,
    'target': target_array,
    'feature_names': feature_names,
    'participants': participants,
    'task': task
}

data_dict

{'data': array([[1.18857100e-01, 1.71499300e-01, 3.82434308e+02, ...,
         4.67743047e+04, 5.17347900e-01, 9.58791500e-01],
        [2.25142900e-01, 1.73505000e-01, 3.90361005e+02, ...,
         3.16159256e+04, 5.27694200e-01, 9.45842000e-01],
        [2.36000000e-01, 2.22698600e-01, 4.34669530e+02, ...,
         2.64352380e+04, 5.20547800e-01, 9.12525700e-01],
        ...,
        [1.19555600e-01, 1.22848900e-01, 1.59285907e+02, ...,
         1.25266151e+04, 5.43171200e-01, 9.99821100e-01],
        [1.56750000e+00, 9.80965500e-01, 2.89913012e+03, ...,
         5.10806271e+04, 4.72444400e-01, 1.07019450e+00],
        [2.16500000e-01, 1.61567600e-01, 3.19002015e+02, ...,
         1.69910575e+04, 5.20748900e-01, 9.41270500e-01]]),
 'target': array([0, 0, 0, ..., 0, 1, 0]),
 'feature_names': ['ImP_exp_diff_mean',
  'ImP_exp_dur_sd',
  'ImP_RRV_RMSSD',
  'ImP_insp_flow_sd',
  'ImP_ie_ratio_sd',
  'ImP_duty_cycle_mean',
  'ImP_insp_amp_sd',
  'ImP_exp_amp_sd',
  'ImP_RSP_Symmetry_PeakTr

In [4]:
#running the nested cross-validation for the Impedance (using VU-AMS 5fs) method with Random Forest
from sklearn.metrics import accuracy_score, precision_score, recall_score
import xgboost as xgb

# random seed for reproducibility
random_seed = 1
np.random.seed(random_seed)

X = data_dict['data']
Y = data_dict['target']
groups = data_dict['participants']

# setting up the parameter grid: the combinations of these will be tried out for ever outer loop's inner loop
param_grid = {
    'n_estimators': [25, 50, 100, 150],
    'max_features': ['sqrt', 'log2', None],
    'max_depth': [3, 6, 9],
    'max_leaf_nodes': [3, 6, 9],
}

rfc = RandomForestClassifier(random_state=random_seed, class_weight="balanced")

logo = LeaveOneGroupOut()


# defining the techniques to use for the inner and outer loops
inner_cv = KFold(n_splits=5, shuffle=True, random_state=1)
outer_cv = logo.split(X, Y, groups)

scoring = {
    'accuracy': make_scorer(accuracy_score),
    'precision': make_scorer(precision_score, average='weighted'),
    'recall': make_scorer(recall_score, average='weighted'),
    'f1_score': make_scorer(f1_score, average='macro'),
    'roc_auc': make_scorer(roc_auc_score, needs_proba=True, average='weighted')
}

# performing nested cross-validation
clf = GridSearchCV(estimator=rfc, param_grid=param_grid, cv=inner_cv)
cv_dic = cross_validate(clf, X, Y, cv=outer_cv, scoring=scoring, return_estimator=True, return_train_score=False)
mean_acc_score = cv_dic['test_accuracy'].mean()
mean_prec_score = cv_dic['test_precision'].mean()
mean_rec_score = cv_dic['test_recall'].mean()
mean_f1_score = cv_dic['test_f1_score'].mean()
mean_roc_auc = cv_dic['test_roc_auc'].mean()  # Mean ROC AUC score


print('nested_accuracy_scores:   ', cv_dic['test_accuracy'])
print('mean score:            {0:.4f}'.format(mean_acc_score))

print('nested_precision_scores:   ', cv_dic['test_precision'])
print('mean score:            {0:.4f}'.format(mean_prec_score))

print('nested_recall_scores:   ', cv_dic['test_recall'])
print('mean score:            {0:.4f}'.format(mean_rec_score))

print('nested_f1_scores:   ', cv_dic['test_f1_score'])
print('mean score:            {0:.4f}'.format(mean_f1_score))

print('nested_roc_auc_scores:   ', cv_dic['test_roc_auc'])
print('mean score:            {0:.4f}'.format(mean_roc_auc))

nested_accuracy_scores:    [0.93939394 0.95454545 0.86363636 0.81818182 0.92424242 0.92424242
 0.86363636 0.90909091 0.89393939 0.68181818 0.71212121 0.77272727
 0.90909091 0.90909091 1.         0.84848485 0.87878788 0.86363636
 0.92424242 0.60606061 0.8030303  0.92424242 0.89393939 0.89393939
 0.86363636 0.77272727 0.86363636 0.89393939 0.86363636 0.8030303
 0.87878788 0.92424242 0.93939394 0.98484848 0.96969697 0.71212121
 0.83333333 0.75757576 0.84848485]
mean score:            0.8621
nested_precision_scores:    [0.94008264 0.95757576 0.86694493 0.82255245 0.93230174 0.92981093
 0.87782767 0.91223776 0.9000259  0.74514954 0.79435379 0.80991736
 0.91223776 0.90909091 1.         0.8486166  0.89747475 0.90082645
 0.93230174 0.65959596 0.82815249 0.93230174 0.89525499 0.90756587
 0.87024087 0.77093483 0.88797654 0.89337255 0.87782767 0.85479682
 0.89747475 0.92398198 0.94213287 0.98545455 0.96969697 0.73841642
 0.87052342 0.83373206 0.86767677]
mean score:            0.8794
nested_recal

In [5]:
sensitivity_scores = []
specificity_scores = []

# iteration over the outer folds to calculate average sensitivity and specificity
for i, (train_index, test_index) in enumerate(logo.split(X, Y, groups)):
    print(f"Fold {i + 1}:")

    # getting the trained estimator for the present fold
    estimator = cv_dic['estimator'][i]
    
    # predicting on the test set
    y_pred = estimator.predict(X[test_index])
    
    # creating confusion matrix
    tn, fp, fn, tp = confusion_matrix(Y[test_index], y_pred).ravel()
    
    # calculating the sensitivity and specificity
    sensitivity = tp / (tp + fn)
    specificity = tn / (tn + fp)
    
    print(f"Sensitivity: {sensitivity:.4f}")
    print(f"Specificity: {specificity:.4f}")
    print("=" * 30)
    
    # store the sensitivity and specificity scores
    sensitivity_scores.append(sensitivity)
    specificity_scores.append(specificity)

    
    # calculate and print the averages of the sensitivity and specificity scores previously calculated
average_sensitivity = np.mean(sensitivity_scores)
average_specificity = np.mean(specificity_scores)

print("Average Sensitivity:", average_sensitivity)
print("Average Specificity:", average_specificity)

Fold 1:
Sensitivity: 0.8750
Specificity: 0.9762
Fold 2:
Sensitivity: 0.8750
Specificity: 1.0000
Fold 3:
Sensitivity: 0.7083
Specificity: 0.9524
Fold 4:
Sensitivity: 0.7917
Specificity: 0.8333
Fold 5:
Sensitivity: 0.7917
Specificity: 1.0000
Fold 6:
Sensitivity: 0.9583
Specificity: 0.9048
Fold 7:
Sensitivity: 0.9167
Specificity: 0.8333
Fold 8:
Sensitivity: 0.9167
Specificity: 0.9048
Fold 9:
Sensitivity: 0.9167
Specificity: 0.8810
Fold 10:
Sensitivity: 0.8333
Specificity: 0.5952
Fold 11:
Sensitivity: 0.9167
Specificity: 0.5952
Fold 12:
Sensitivity: 0.8750
Specificity: 0.7143
Fold 13:
Sensitivity: 0.9167
Specificity: 0.9048
Fold 14:
Sensitivity: 0.8750
Specificity: 0.9286
Fold 15:
Sensitivity: 1.0000
Specificity: 1.0000
Fold 16:
Sensitivity: 0.7083
Specificity: 0.9286
Fold 17:
Sensitivity: 0.9583
Specificity: 0.8333
Fold 18:
Sensitivity: 1.0000
Specificity: 0.7857
Fold 19:
Sensitivity: 0.7917
Specificity: 1.0000
Fold 20:
Sensitivity: 0.7083
Specificity: 0.5476
Fold 21:
Sensitivity: 0.8750


In [6]:
#print all performance scores that might be of interest
import scipy.stats

def calculate_stats(data):
    mean = np.mean(data)
    sd = np.std(data, ddof=1)  # using sample standard deviation (N-1)
    ci = scipy.stats.norm.interval(0.95, loc=mean, scale=sd/np.sqrt(len(data)))
    min_val = np.min(data)
    max_val = np.max(data)
    return mean, sd, ci, min_val, max_val

keys = ['test_accuracy', 'test_precision', 'test_recall', 'test_f1_score', 'test_roc_auc', 'sensitivity_scores', 'specificity_scores']

for key in keys:
    if key in cv_dic:
        data = cv_dic[key]
    elif key == 'sensitivity_scores':
        data = sensitivity_scores
    elif key == 'specificity_scores':
        data = specificity_scores

    stats = calculate_stats(data)
    print(f"{key}: Mean={stats[0]}, SD={stats[1]}, 95% CI={stats[2]}, Min={stats[3]}, Max={stats[4]}")


test_accuracy: Mean=0.8620823620823622, SD=0.08534224014609888, 95% CI=(0.835298091360543, 0.8888666328041814), Min=0.6060606060606061, Max=1.0
test_precision: Mean=0.8793958662381283, SD=0.0700534187337288, 95% CI=(0.8574099220076858, 0.9013818104685707), Min=0.6595959595959595, Max=1.0
test_recall: Mean=0.8620823620823622, SD=0.08534224014609888, 95% CI=(0.835298091360543, 0.8888666328041814), Min=0.6060606060606061, Max=1.0
test_f1_score: Mean=0.8558004123091945, SD=0.0854672415632061, 95% CI=(0.8289769104658264, 0.8826239141525625), Min=0.6027777777777777, Max=1.0
test_roc_auc: Mean=0.9467974155474155, SD=0.0554213293984788, 95% CI=(0.9294036854574768, 0.9641911456373542), Min=0.7594246031746031, Max=1.0
sensitivity_scores: Mean=0.8814102564102564, SD=0.08892349894132379, 95% CI=(0.8535020240378877, 0.9093184887826251), Min=0.6666666666666666, Max=1.0
specificity_scores: Mean=0.851037851037851, SD=0.12247180110952578, 95% CI=(0.8126006378701311, 0.8894750642055709), Min=0.547619047

In [7]:
# IF this is the best-performing nested CV for the method: print best hyperparameters for each outer loop
for fold_idx, estimator in enumerate(cv_dic['estimator']):
    print(f'Fold {fold_idx + 1} - Best hyperparameters: {estimator.best_params_}')

Fold 1 - Best hyperparameters: {'max_depth': 6, 'max_features': None, 'max_leaf_nodes': 9, 'n_estimators': 25}
Fold 2 - Best hyperparameters: {'max_depth': 6, 'max_features': None, 'max_leaf_nodes': 9, 'n_estimators': 25}
Fold 3 - Best hyperparameters: {'max_depth': 6, 'max_features': None, 'max_leaf_nodes': 9, 'n_estimators': 25}
Fold 4 - Best hyperparameters: {'max_depth': 6, 'max_features': None, 'max_leaf_nodes': 9, 'n_estimators': 100}
Fold 5 - Best hyperparameters: {'max_depth': 6, 'max_features': None, 'max_leaf_nodes': 9, 'n_estimators': 25}
Fold 6 - Best hyperparameters: {'max_depth': 6, 'max_features': None, 'max_leaf_nodes': 9, 'n_estimators': 50}
Fold 7 - Best hyperparameters: {'max_depth': 6, 'max_features': None, 'max_leaf_nodes': 9, 'n_estimators': 50}
Fold 8 - Best hyperparameters: {'max_depth': 6, 'max_features': None, 'max_leaf_nodes': 9, 'n_estimators': 50}
Fold 9 - Best hyperparameters: {'max_depth': 6, 'max_features': None, 'max_leaf_nodes': 9, 'n_estimators': 50}


In [8]:
# IF this is the best-performing nested CV for the method: print mode of hyperparameters

from collections import defaultdict, Counter

# initialize a dictionary to contain all hyperparameters and their values across loops
hyperparameters = defaultdict(list)

# extracting hyperparameters of each outer loop
for estimator in cv_dic['estimator']:
    for param, value in estimator.best_params_.items():
        hyperparameters[param].append(value)

# count the most frequent value for each hyperparameter
for param, values in hyperparameters.items():
    most_common_value, count = Counter(values).most_common(1)[0]  # Get the most frequent value
    print(f'Hyperparameter: {param} - Most frequent value: {most_common_value} (Count: {count})')


Hyperparameter: max_depth - Most frequent value: 6 (Count: 39)
Hyperparameter: max_features - Most frequent value: None (Count: 38)
Hyperparameter: max_leaf_nodes - Most frequent value: 9 (Count: 39)
Hyperparameter: n_estimators - Most frequent value: 25 (Count: 16)
